In [37]:
import os
from pathlib import Path 
import pandas as pd, numpy as np, datetime 
import matplotlib.pyplot as plt, seaborn as sns 
import plotly.graph_objects as go
from utils.fipi_lipi_processor import FIPIDataProcessor
from config.config import (FIPI_DIR, LIPI_DIR, PROCESSED_FIPI_LIPI_DIR, STOCKS_DIR, RAW_STOCKS_DIR, QUOTATION_DATA_DIR)
from utils.fipi_lipi_preprocessor_v2 import FIPIDataProcessor
from utils.db_connection_v2 import DatabaseConnection
from config.db_config import db_config
from datetime import date, datetime


from scrapper.quotationextraction import PDFExtraction as qe
from scrapper.scrapper import Scrapper as sp
from scrapper.datareader import DataReader as dr


import pymysql


sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (18, 6)
plt.rcParams['font.size'] = 14
plt.rcParams['figure.dpi'] = 300
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [21]:
file_name = '01-01-2019lipi.csv'

date_str = file_name.split('lipi')[0]

date_fipi = datetime.strptime(date_str, '%d-%m-%Y')
print(date_fipi)

2019-01-01 00:00:00


In [24]:
directory = "data/lipi"  # Directory containing your FIPI CSV files
processor = FIPIDataProcessor(directory, 'lipi')

# Process and get the cleaned data
cleaned_data = processor.process_files()

Scanning directory: 2019 : data\lipi\2019
Scanning directory: 2020 : data\lipi\2020
Scanning directory: 2021 : data\lipi\2021
Scanning directory: 2022 : data\lipi\2022
Scanning directory: 2023 : data\lipi\2023
Scanning directory: 2024 : data\lipi\2024


d:\fund_tracker_psx\utils\fipi_lipi_preprocessor_v2.py:134: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  # df_clean['FUTURE_CONTRACT_MONTH'] = df_clean['FUTURE_CONTRACT_MONTH'].where(pd.notnull(df_clean['FUTURE_CONTRACT_MONTH']), None)
d:\fund_tracker_psx\utils\fipi_lipi_preprocessor_v2.py:134: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  # df_clean['FUTURE_CONTRACT_MONTH'] = df_clean['FUTURE_CONTRACT_MONTH'].where(pd.notnull(df_clean['FUTURE_CONTRACT_MONTH']), None)
d:\fund_tracker_psx\utils\fipi_lipi_preprocessor_v2.py:134: FutureWarning: Downcasting

In [28]:
lipi_type = cleaned_data['MARKET TYPE'].unique()

In [53]:
len(cleaned_data)

215314

In [39]:
lipi_data = cleaned_data.head()

In [ ]:
db = DatabaseConnection(**db_config)
cursor = db.connect()
cursor.execute('SELECT DATABASE();')

Connected to the database successfully!


In [37]:
query = f"""
INSERT INTO fipi_lipi_market_types(
market_type_name, market_type_description, market_type_status, 
fipi_lipi_market_type_added_date, fipi_lipi_market_type_added_by, fipi_lipi_market_type_modified_date, fipi_lipi_market_type_modified_by
) VALUES (%s,%s,%s,%s,%s,%s,%s)

"""
for data in lipi_type:
    values = (
        data,
        f"{data}_lipi",
        1,
        date.today(),
        1000,
        date.today(),
        1000
    )
    # print(values)
    cursor.execute(query, values)

db.commit()

Connection lost. Reconnecting...
Transaction committed successfully!


In [44]:
sector_type = f""" SELECT fipi_lipi_sector_id, fipi_lipi_sector_code FROM fipi_lipi_sector """
cursor.execute(sector_type)
sector_type_data = cursor.fetchall()

sector_type_mapping = {market_id[1]: market_id[0] for market_id in sector_type_data}

Connection lost. Reconnecting...


In [45]:
market_type = f""" SELECT fipi_lipi_market_type_id, market_type_name FROM fipi_lipi_market_types """
cursor.execute(market_type)
market_type_data = cursor.fetchall()

market_type_mapping = {market_id[1]: market_id[0] for market_id in market_type_data}

In [46]:
client_type = f""" SELECT fipi_lipi_client_type_id, client_type_name FROM fipi_lipi_client_types """
cursor.execute(client_type)
client_type_data = cursor.fetchall()

client_type_mapping = {market_id[1]: market_id[0] for market_id in client_type_data}

In [ ]:
client_id = f""" SELECT fipi_lipi_client_id, fipi_lipi_market_type_id, fipi_lipi_sector_id, fipi_lipi_sector_id FROM fipi_lipi_clients """
cursor.execute(client_id)
client_id_data = cursor.fetchall()

Connection lost. Reconnecting...


In [60]:
client_id_data

((1, 2, 10, 10),
 (2, 3, 8, 8),
 (3, 3, 4, 4),
 (4, 3, 2, 2),
 (5, 3, 6, 6),
 (6, 3, 5, 5),
 (7, 3, 7, 7),
 (8, 3, 8, 8),
 (9, 3, 9, 9),
 (10, 3, 10, 10),
 (11, 3, 3, 3),
 (12, 3, 11, 11),
 (13, 3, 4, 4),
 (14, 1, 4, 4),
 (15, 2, 2, 2),
 (16, 2, 6, 6),
 (17, 2, 5, 5),
 (18, 2, 7, 7),
 (19, 2, 8, 8),
 (20, 2, 9, 9),
 (21, 2, 10, 10),
 (22, 2, 3, 3),
 (23, 2, 11, 11),
 (24, 2, 4, 4),
 (25, 1, 10, 10),
 (26, 2, 2, 2),
 (27, 2, 6, 6),
 (28, 2, 7, 7),
 (29, 2, 8, 8),
 (30, 2, 9, 9),
 (31, 2, 11, 11),
 (32, 2, 4, 4),
 (33, 3, 6, 6),
 (34, 2, 2, 2),
 (35, 2, 3, 3),
 (36, 3, 2, 2),
 (37, 3, 3, 3),
 (38, 2, 6, 6),
 (39, 2, 5, 5),
 (40, 2, 10, 10),
 (41, 2, 4, 4),
 (42, 2, 5, 5),
 (43, 3, 5, 5),
 (44, 1, 2, 2),
 (45, 1, 9, 9),
 (46, 1, 10, 10),
 (47, 2, 8, 8),
 (48, 3, 2, 2),
 (49, 3, 7, 7),
 (50, 1, 2, 2),
 (51, 1, 6, 6),
 (52, 1, 7, 7),
 (53, 1, 8, 8),
 (54, 1, 4, 4),
 (55, 3, 10, 10),
 (56, 1, 6, 6),
 (57, 1, 7, 7),
 (58, 1, 8, 8),
 (59, 1, 3, 3),
 (60, 1, 11, 11),
 (61, 1, 9, 9),
 (62, 2, 9,

In [61]:
lipi_data

,Date,CLIENT TYPE,SEC CODE,SECTOR NAME,MARKET TYPE,BUY VOLUME,BUY VALUE,SELL VOLUME,SELL VALUE,NET VOLUME,NET VALUE,USD,market_type_name,FUTURE_CONTRACT_MONTH,Year,Month
0,2019-01-01,BANKS / DFI,S0028,Technology and Communication,FUT,0.0,0.0,-165000.0,-3772320.0,-165000.0,-3772320.0,-27138.0,FUT,0,2019,1
1,2019-01-01,BANKS / DFI,S1111,All other Sectors,FUT,9000.0,269905.0,-160000.0,-4747120.0,-151000.0,-4477215.0,-32210.0,FUT,0,2019,1
2,2019-01-01,BANKS / DFI,S1111,All other Sectors,ODL,0.0,0.0,-400.0,-25663.0,-400.0,-25663.0,-184.0,ODL,0,2019,1
3,2019-01-01,BANKS / DFI,S0005,Cement,REG,175000.0,14468955.0,-10000.0,-4442625.0,165000.0,10026330.0,72131.0,REG,0,2019,1
4,2019-01-01,BANKS / DFI,S0008,Food and Personal Care Products,REG,0.0,0.0,-50000.0,-1457500.0,-50000.0,-1457500.0,-10485.0,REG,0,2019,1


In [47]:
print(client_type_mapping, '\n', market_type_mapping, '\n', sector_type_mapping)

{'BANKS / DFI': 1, 'BROKER PROPRIETARY TRADING': 2, 'COMPANIES': 3, 'INDIVIDUALS': 4, 'INSURANCE COMPANIES': 5, 'MUTUAL FUNDS': 6, 'NBFC': 7, 'OTHER ORGANIZATION': 8, 'FOREIGN CORPORATES ': 9, 'FOREIGN INDIVIDUAL': 10, 'OVERSEAS PAKISTANI': 11} 
 {'OFF-MARKET': 1, 'REGULAR': 2, 'FUTURE CONTRACT': 3, 'FUT': 4, 'ODL': 5, 'REG': 6, 'OMKT': 7, 'BNB': 8, 'GEM': 9} 
 {'S0036': 1, 'S0005': 2, 'S0028': 3, 'S1111': 4, 'S0008': 5, 'S0007': 6, 'S0019': 7, 'S0020': 8, 'S0023': 9, 'S0026': 10, 'S0029': 11}


In [65]:
import pandas as pd
from datetime import date

# Pre-load existing combinations into a dictionary
def preload_existing_combinations():
    try:
        cursor = db.connect()
        query = """
            SELECT fipi_lipi_client_id, fipi_lipi_client_type_id, 
                   fipi_lipi_market_type_id, fipi_lipi_sector_id 
            FROM fipi_lipi_clients
        """
        cursor.execute(query)
        data = cursor.fetchall()

        # Create a dictionary for quick lookup
        return {
            (row[1], row[2], row[3]): row[0]  # Key: (client_type_id, market_type_id, sector_type_id), Value: client_id
            for row in data
        }
    except Exception as e:
        print(f"Error loading existing combinations: {e}")
        return {}

# Insert new combination into the database and update the dictionary
def insert_new_combination(cursor, client_type, market_type_name, sector_type):
    try:
        insert_query = """
            INSERT INTO fipi_lipi_clients (
                fipi_lipi_client_type_id, fipi_lipi_market_type_id, 
                fipi_lipi_sector_id, fipi_lipi_client_added_date, 
                fipi_lipi_client_added_by, fipi_lipi_client_modified_date, 
                fipi_lipi_client_modified_by
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(
            insert_query,
            (client_type, market_type_name, sector_type, date.today(), 1000, date.today(), 1000)
        )
        db.commit()
        return cursor.lastrowid
    except Exception as e:
        print(f"Error inserting new combination: {e}")
        raise

# Process the DataFrame with preloaded combinations using executemany()
def process_data_with_cache(lipi_data):
    try:
        cursor = db.connect()
        existing_combinations = preload_existing_combinations()
        print(f"Loaded {len(existing_combinations)} existing combinations into memory.")

        # Prepare data for bulk insertion
        bulk_data = []
        for index, row in lipi_data.iterrows():
            client_type_id = client_type_mapping.get(row["CLIENT TYPE"])
            market_type_id = market_type_mapping.get(row["market_type_name"])
            sector_type_id = sector_type_mapping.get(row["SEC CODE"])

            if not client_type_id or not market_type_id or not sector_type_id:
                print(f"Mapping not found for row: {row}")
                continue

            # Check if the combination exists in the dictionary
            combination_key = (client_type_id, market_type_id, sector_type_id)
            if combination_key in existing_combinations:
                fipi_lipi_client_id = existing_combinations[combination_key]
            else:
                # Insert new combination and update the dictionary
                fipi_lipi_client_id = insert_new_combination(cursor, client_type_id, market_type_id, sector_type_id)
                existing_combinations[combination_key] = fipi_lipi_client_id

            # Prepare data for bulk insertion
            bulk_data.append((
                fipi_lipi_client_id,
                row["BUY VOLUME"], row["BUY VALUE"], row["SELL VOLUME"],
                row["SELL VALUE"], row["NET VOLUME"], row["NET VALUE"], 
                row["USD"], row['FUTURE_CONTRACT_MONTH'], row["Month"], 
                row["Year"], row["Date"], date.today(), 1000, 
                date.today(), 1000
            ))

        # Insert data in bulk using executemany()
        insert_query = """
            INSERT INTO lipi (
                fipi_lipi_client_id, lipi_buy_volume, lipi_buy_value, 
                lipi_sell_volume, lipi_sell_value, lipi_net_volume, 
                lipi_net_value, lipi_usd, lipi_contract_month, 
                lipi_month, lipi_year, lipi_date, lipi_added_date, 
                lipi_added_by, lipi_modified_date, lipi_modified_by
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.cursor.executemany(insert_query, bulk_data)

        db.commit()
        print("All changes committed successfully!")
    except Exception as e:
        print(f"Error during data processing: {e}")
        db.rollback()

# Example usage
process_data_with_cache(cleaned_data)

Error executing query: (2006, "MySQL server has gone away (ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))")
Error loading existing combinations: (2006, "MySQL server has gone away (ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))")
Loaded 0 existing combinations into memory.
Connection lost. Reconnecting...
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed successfully!
Transaction committed

In [161]:
class scanner:
    def __init__(self, directory) -> None:
            self.directory = directory

    def filter_scanner(self):
        all_csv_files = []
        with os.scandir(self.directory) as files:
             csv_files = [file.path for file in files if file.is_file() and file.name.endswith('.csv')]
             all_csv_files.extend(csv_files)

             return all_csv_files
        

bs_class = scanner(Path(RAW_STOCKS_DIR)).filter_scanner()


In [ ]:
query = f"""
SELECT stock_symbol, stock_id FROM exchange_sector_stock;
"""
db = DatabaseConnection(**db_config)
cursor = db.connect()
cursor.execute(query)
results = cursor.fetchall()
stocks_mapping = {symbol[0]:symbol[1] for symbol in results}
print(stocks_mapping)

Connected to the database successfully!
(('786', 1), ('AABS', 2), ('AAL', 3), ('AASM', 4), ('AATM', 5), ('ABL', 6), ('ABOT', 7), ('ABSON', 8), ('ACIETF', 9), ('ACPL', 10), ('ADAMS', 11), ('ADMM', 12), ('AGHA', 13), ('AGIC', 14), ('AGIL', 15), ('AGL', 16), ('AGLNCPS', 17), ('AGP', 18), ('AGSML', 19), ('AGTL', 20), ('AHCL', 21), ('AHL', 22), ('AHTM', 23), ('AICL', 24), ('AIRLINK', 25), ('AKBL', 26), ('AKDHL', 27), ('AKDSL', 28), ('AKGL', 29), ('ALAC', 30), ('ALIFE', 31), ('ALNRS', 32), ('ALTN', 33), ('AMBL', 34), ('AMSL', 35), ('AMTEX', 36), ('ANL', 37), ('ANLNV', 38), ('ANLPS', 39), ('ANNT', 40), ('ANSM', 41), ('ANTM', 42), ('APL', 43), ('APOT', 44), ('ARCTM', 45), ('ARPAK', 46), ('ARPL', 47), ('ARUJ', 48), ('ASC', 49), ('ASHT', 50), ('ASIC', 51), ('ASL', 52), ('ASLCPS', 53), ('ASLPS', 54), ('ASTL', 55), ('ASTM', 56), ('ATBA', 57), ('ATIL', 58), ('ATLH', 59), ('ATRL', 60), ('AVN', 61), ('AWTX', 62), ('AZMT', 63), ('BAFL', 64), ('BAFS', 65), ('BAHL', 66), ('BAPL', 67), ('BATA', 68), ('BC

In [ ]:
dataframes = []

for file in bs_class:
    df = pd.read_csv(file)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

# combined_df.to_csv("combined_stocks.csv", index=False)
combined_df['Symbol'] = combined_df['Symbol'].replace(stocks_mapping).fillna(-1)

combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Month'] = combined_df['Date'].dt.month
combined_df['Day'] = combined_df['Date'].dt.dayofweek
combined_df['Quarter'] = combined_df['Date'].dt.quarter
filtered_df = combined_df[combined_df['Symbol'].isin(stocks_mapping.values())]
filtered_df['ldcp'] = filtered_df['close'].shift(1)
filtered_df['daily_added_date'] = date.today()
filtered_df['daily_added_by'] = 1000  # Use your user ID or default value
filtered_df['daily_modified_date'] = date.today()
filtered_df['daily_modified_by'] = 1000

filtered_df['daily_data_date'] = pd.to_datetime(filtered_df['daily_data_date'])

C:\Users\razaa\AppData\Local\Temp\ipykernel_31016\1366050395.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['daily_added_date'] = date.today()
C:\Users\razaa\AppData\Local\Temp\ipykernel_31016\1366050395.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['daily_added_by'] = 1000  # Use your user ID or default value
C:\Users\razaa\AppData\Local\Temp\ipykernel_31016\1366050395.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [271]:
# Rename the columns in the DataFrame
filtered_df = filtered_df.rename(columns={
    'Symbol': 'stock_id',
    'Date': 'daily_data_date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume',
    'Year': 'daily_data_year',
    'Month': 'daily_data_month',
    'Day': 'daily_data_day',
    'Quarter': 'daily_data_quarter',
    'daily_added_date': 'daily_data_added_date',
    'daily_added_by': 'daily_data_added_by',
    'daily_modified_date':'daily_data_modified_date',
    'daily_modified_by':'daily_data_modified_by'

})


In [295]:
from datetime import date
def bulk_insert_data(filtered_df, db_config):
    try:

        bulk_data = []
        for index, row in filtered_df.iterrows():
            bulk_data.append((
                row["stock_id"],
                row["open"], row["high"], row["low"], row["close"], row['ldcp'],
                row["volume"], row["daily_data_date"], row["daily_data_year"],
                row["daily_data_month"], row["daily_data_day"], row["daily_data_quarter"],
                date.today(), 1000, date.today(), 1000  
            ))

        insert_query = f"""
            INSERT INTO stock_daily_data (
                stock_id, open, high, low, close, volume, ldcp,
                daily_data_date, daily_data_year, daily_data_month,
                daily_data_day, daily_data_quarter, daily_data_added_date,
                daily_data_added_by, daily_data_modified_date, daily_data_modified_by
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
        """
        db = DatabaseConnection(**db_config)
        cursor = db.connect()
        cursor.cursor.executemany(insert_query, bulk_data)

        # Commit the transaction
        db.commit()
        print("All changes committed successfully!")

    except Exception as e:
        print(f"Error during bulk insertion: {e}")
        db.rollback()

# Example Usage
bulk_insert_data(filtered_df, db_config)


Connected to the database successfully!
Transaction committed successfully!
All changes committed successfully!


In [34]:
import re
import logging
from pathlib import Path
from zipfile import ZipFile
from datetime import datetime, date

import pdfplumber
import pandas as pd

import scrapper as sp


class DataReader:
    def __init__(self) -> None:
        self.data_folder = Scrapper().create_or_get_folder()

    def get_files(self) -> dict[str, list[Path]]:
        dictionary = Path(self.data_folder)

        files_by_extension = {}
        # iter over all the files in the directory
        for file in dictionary.iterdir():
            if file.is_file():
                file_extension = file.suffix.lower()
                # Checking if the extension is in the dictionary
                if file_extension in files_by_extension:
                    files_by_extension[file_extension].append(file)
                else:
                    files_by_extension[file_extension] = [file]

        return files_by_extension

    def get_file_reader(self):
        readers: list = []
        for file_extension, files in self.get_files().items():
            for file in files:
                stem_file_name = file.stem
                stem_file_name = re.sub('[^A-Za-z_]', '', stem_file_name)

                if file_extension == '.csv':
                    file_path = file
                    reader = pd.read_csv
                    readers.append(reader)
                elif file_extension == '.xls':
                    if stem_file_name.startswith("fut"):
                        file_path = file
                        reader = pd.read_excel
                    else:
                        file_path = file
                        reader = pd.read_excel
                    readers.append(reader)
                elif file_extension == '.pdf':
                    file_path = file
                    reader = pdfplumber.open
                    readers.append(reader)
                elif file_extension == '.z':
                    self.read_zip_file()
                    file_path = file
                    reader = pd.read_csv
                    readers.append(reader)

        return readers

    def get_file_path(self):
        file_path = []
        for file_extension, files in self.get_files().items():
            for file in files:
                file_path.append(file)
        return file_path

    def read_zip_file(self):
        try:
            zip_files = self.get_files().get('.z', [])
            if not zip_files:
                raise FileNotFoundError("Zip file not found")

            for zip_file in zip_files:
                with ZipFile(zip_file) as zp:
                    zp.extractall(self.data_folder)
                    extracted_file_name = "closing11.lis"
                    extracted_file_path = self.data_folder / extracted_file_name
                    print(f"File '{zip_file.name}' extracted successfully")

            return extracted_file_path

        except FileNotFoundError as e:
            logging.warning(str(e))


class DataPreprocessor:

    @staticmethod
    def get_market_summary(file_path) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Read and preprocess market summary data from a CSV file.

        Parameters:
        - file_path (Path): The path to the CSV file.

        Returns:
        - tuple[pd.DataFrame, pd.DataFrame]: Two DataFrames representing ready market and future market data.
        """
        try:
            df = pd.read_csv(file_path, header=None, sep='|')
            if df.empty:
                raise pd.errors.EmptyDataError("CSV file is empty")

            df = df.iloc[:, :10]
            df.columns = ['date', 'Symbol', 'Sector Code', 'Name', 'open', 'high', 'low', 'close', 'volume', 'LCDP']
            df['date'] = pd.to_datetime(df['date'])

            ready_market = df[(df['Sector Code'] != 40) & (df['Sector Code'] != 41)]
            future_market = df[(df['Sector Code'] == 40) | (df['Sector Code'] == 41)]

            return ready_market, future_market
        except pd.errors.EmptyDataError as e:
            logging.warning(f"EmptyDataError: {e}")
        except pd.errors.ParserError as e:
            logging.warning(f"ParserError: {e}")
        except Exception as e:
            logging.warning(f"An unexpected error occurred: {e}")

    @staticmethod
    def get_indhist(file_path):
        pass

    @staticmethod
    def get_omts(file_path):
        """
            This method receives a CSV file path and splits the data into two separate pd.DataFrames.

            Parameters:
                file_path (str): CSV file path to read.

            Returns:
                df1: The first processed DataFrame containing off-market BROKER TO BROKER TRADES.
                df2: The second processed DataFrame containing CROSS TRANSACTIONS BETWEEN CLIENT TO CLIENT & FINANCIAL INSTITUTIONS
            """
        try:
            df = pd.read_csv(file_path, skip_blank_lines=True, skiprows=4)
            # Find the index of the first row where any column is NaN
            separator_index = df[df.isnull().any(axis=1)].index[0]
            # Slice the DataFrame into two separate DataFrames using this index
            df1 = df.loc[: separator_index - 1].copy()
            df2 = df.loc[separator_index + 2:].copy()  # Skip the first row directly
            df1.reset_index(
                drop=True, inplace=True
            )  # reset the index to 0,1,2… after slicing
            df2.reset_index(drop=True, inplace=True)
            df1.columns = df1.columns.str.strip()
            df2.columns = df2.columns.str.strip()
            # split the 'MEMBER CODE' column literal string search
            df1[["0", "BUYER", "2", "SELLER"]] = df1["MEMBER CODE"].str.split(
                " ", expand=True
            )
            # Drop the unnecessary columns
            df1.drop(columns=["MEMBER CODE", "0", "2"], inplace=True)
            df1 = df1[
                [
                    "Date",
                    "SETTLEMENT DATE",
                    "BUYER",
                    "SELLER",
                    "SYMBOL CODE",
                    "COMPANY",
                    "TURNOVER",
                    "RATE",
                    "VALUES",
                ]
            ]  # reorder the columns
            return df1, df2
        except Exception as e:
            raise Exception(f"Error: {e}")

    @staticmethod
    def get_open_interest(file_path):
        """
        Extracts datatable from an `Excel file .xls` and returns a pandas DataFrame.

        Returns:
            pandas.DataFrame: The DataFrame containing the fetched data.
        """
        try:
            sheet_names = pd.ExcelFile(file_path).sheet_names
            df = pd.read_excel(file_path, skiprows=2, sheet_name=sheet_names[1], header=0, index_col=0)
            df.columns = ['Symbol', 'Category', 'OI Contract', 'OI Volume', 'OI Value', 'FF of Script', "% FF"]
            df = df.iloc[3:, :].dropna()

            return df
        except Exception as e:
            print(f"Error Message: {e}")


if __name__ == '__main__':
    dd = DataPreprocessor()
    df1, df2 = dd.get_market_summary(DataReader().read_zip_file())
    print(df1)


File 'mkt_summary2024-12-10.Z' extracted successfully
          date        Symbol  Sector Code                  Name      open  \
0   2024-12-10           786          813        786 Invest Ltd      7.29   
1   2024-12-10          AABS          826        Al-Abbas Sugar    681.01   
2   2024-12-10          AATM          830    Ali Asghar Textile     64.49   
3   2024-12-10           ABL          807       Allied Bank Ltd    145.95   
4   2024-12-10          ABOT          823           Abbott Lab.   1120.00   
5   2024-12-10        ACIETF          837      Alfalah Consumer     14.75   
6   2024-12-10          ACPL          804         Attock Cement    214.00   
7   2024-12-10         ADAMS          826            Adam Sugar     69.98   
8   2024-12-10          ADMM          829        Artistic Denim     50.00   
9   2024-12-10          AGHA          808       Agha Steel Ind.     11.10   
13  2024-12-10          AGIC          812        Ask.Gen.Insur.     29.25   
14  2024-12-10        

In [ ]:
import logging
from tqdm import tqdm
from glob import glob
from datetime import date

from scrapper import scrapper as sp
from scrapper import datareader as dr
from scrapper import quotationextraction as qe


class Execution:
    def __init__(self, target_date: str = str(date.today())) -> None:
        """
        Initializes the object with a data reader and sets the file path attribute.
        """
        self.date = target_date
        self.download_directory = sp.Scrapper().create_or_get_folder()
        if not self._check_existing_files():
            self.today_downloads = sp.Scrapper(self.date).generate_url_links()

        self.file_path: dict = dr.DataReader().get_files()
        self.data_reader = dr.DataPreprocessor()
        self._setup_logging()

    def _setup_logging(self):
        """
        Set up logging for the scraper with the specified storage location.
        """
        logging.basicConfig(
            filename=self.download_directory/"logs"/"scraper.log", level=logging.INFO,
            format="%(asctime)s - %(levelname)s: %(message)s"
        )

    def _check_existing_files(self) -> bool:
        """
        check for existing files in the download directory based on the specified date.
        
        :return: bool indicated whether existing files were found
        """
        file_pattern = f"*{self.date}*"
        existing_files = list(self.download_directory.glob(file_pattern))
        if len(existing_files) > 0:
            logging.info(f"Found {len(existing_files)} existing files")
        else:
            logging.info(f"No existing files found for {self.date}")

        return len(existing_files) > 0

    def _delete_files_in_directory(self):
        """
        Deletes all files in the DataWarehouse directory.
        """
        try:
            for file in self.download_directory.glob("*"):
                if file.is_file():
                    file.unlink()
            logging.info("All files deleted successfully.")
        except Exception as e:
            logging.error(f"Error while deleting files: {e}")

    def execution_by_extension(self):
        """
        Iterates through each file based on its extension and executes the corresponding 
        function to extract and process the data. Returns a list of dataframes.
        """
        df_list = []
        try:
            for extension, files in tqdm(self.file_path.items()):
                logging.info("Executing " + extension)

                if extension == ".z":
                    # Decompress the Unix-documents which is in .Z
                    extracted_file_path = dr.DataReader().read_zip_file()
                    # Calling the right function based on an extension
                    df = self.data_reader.get_market_summary(extracted_file_path)
                    df_list.append(df)

                elif extension == ".csv":
                    for file in files:
                        df = self.data_reader.get_omts(file)
                        df_list.append(df)

                elif extension == ".xls":
                    for file in files:
                        # Stem the file name to get the right file name
                        if file.stem.startswith("fut_opn_int"):
                            df = self.data_reader.get_open_interest(file)
                        else:
                            df = self.data_reader.get_indhist(file)
                        df_list.append(df)
                elif extension == ".pdf":
                    # calling the right function based on an extension and extracting the dict
                    for file in files:
                        df = qe.get_quotes(file)
                        df_list.append(df)
                        
            return df_list
        except Exception as e:
            logging.info(f"Error while executing {extension}: {e}")
            
        # After processing, delete the files
        self._delete_files_in_directory()
        
        # Display output
        return df_list

ee = Execution("2024-12-10")
ee_data = ee.execution_by_extension()

Extracting: 31it [00:19,  1.58it/s]s]
100%|██████████| 1/1 [00:20<00:00, 20.13s/it]

Market       index c_ex_id        Date No. of Listed Companies  \
0       Main Board       1  2024-12-10                     526   
1        GEM Board       1  2024-12-10                       5   

Market Listed Paid up Capital Market Capitalisation Traded Volume  \
0                   1726321 M        13891282177142    1548302320   
1                      4773 M            8676795346          9549   

Market Traded Value Advancers Decliners Neutral Total  
0       68805732964       106       329      34   469  
1            179807         1         2       2     5  
1                                    Company       Place  \
2                    Clover Pakistan Limited     Karachi   
3                  Habib Sugar Mills Limited     Karachi   
4                           MCB Bank Limited      Lahore   
5                        Soneri Bank Limited     Karachi   
6                      Bata Pakistan Limited      Lahore   
7           Tandlianwala Sugar Mills Limited      Lahore   
8    

In [48]:
ee_data

[None]